# Usando a Distância Levenshtein para correção de valores na base de dados
  

<img style="width: 300px" src="https://images.unsplash.com/photo-1535745122259-f1e187953c4c?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxzZWFyY2h8OHx8ZGlzdGFuY2V8ZW58MHwwfDB8fA%3D%3D&auto=format&fit=crop&w=500&q=60" />

Photo by <a href="https://unsplash.com/@mrbrunowolff?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Bruno Wolff</a> on <a href="https://unsplash.com/s/photos/distance?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>
  

Uma das fases mais comuns e custosas da analise de dados é a correção de tabelas de dados com erros de inserção, que leva muitas vezes o trabalho tedioso de substituir cada ocorrência. 

Sabe-se que a distância de Levenshtein (edição) é uma métrica bastante usada para medir similaridade entre cadeias de caracteres e pode ser usada para diversos problemas como correção ortográfica e agrupamentos.

Aqui construi uma forma simples de aplicar a medida ao problema citado acima.

In [ ]:
!pip install editdistance pandas

In [5]:
import editdistance
import pandas as pd

In [7]:
def groupby_min_edit_distance(fact_values: list, 
                             dim_keys: list, 
                             fact_value_label: str="value",
                             dim_key_label: str="key",
                             abbreviation=False):
  edit_distance_collection = list()


  for value in fact_values:
    cross_merge = [{fact_value_label: value, 
                    dim_key_label: key, 
                    "distance": editdistance.eval(value, key)} 
                   for key in keys]
    min_item = min(cross_merge, key=lambda x: x["distance"])
    edit_distance_collection.append(min_item)
  return edit_distance_collection

### Caso de uso: Estados 

Para usar a função basta apenas preencher os parâmetros, sendo que **values** se refere aos valores que desejam ser corrigidos e **keys** a lista dos valores corretos (Na prática recebemos os **values** e especificamos apenas as **keys**) 

In [9]:
values = ["sao paulo", "Sao Paulo", "Bahia", "Bahea", "Paraibama"]
keys = ["São Paulo", "Bahia", "Minas Gerais", "Paraíba"]
reduce_states = groupby_min_edit_distance(fact_values=values, 
                         dim_keys=keys,
                         fact_value_label="Estados",
                         dim_key_label="Correção")
states = pd.json_normalize(reduce_states)
states.head()

,Estados,Correção,distance
0,sao paulo,São Paulo,3
1,Sao Paulo,São Paulo,1
2,Bahia,Bahia,0
3,Bahea,Bahia,1
4,Paraibama,Paraíba,3


### Caso de uso: Turnos

In [10]:
values = ["Vespertino", "Matutino", "Norturno.", "notuno", 'Natutino', "vespertino", "Vespetino"]
keys = ["Matutino", "Vespertino", "Noturno"]
reduce_turn = groupby_min_edit_distance(fact_values=values, 
                         dim_keys=keys,
                         fact_value_label="Turnos",
                         dim_key_label="Correção")
turn = pd.json_normalize(reduce_turn)
turn.head()

,Turnos,Correção,distance
0,Vespertino,Vespertino,0
1,Matutino,Matutino,0
2,Norturno.,Noturno,2
3,notuno,Noturno,2
4,Natutino,Matutino,1


### Caso de uso: Cidades

In [11]:
values = [
    "Phila",
    "Philly",
    "Philadelphia",
    "Philadelphia.",
    "Pittsburgh",
]

keys = ["Philadelphia", "Pittsburgh"]
reduce_city = groupby_min_edit_distance(fact_values=values, 
                         dim_keys=keys,
                         fact_value_label="Cidades",
                         dim_key_label="Correção")
city = pd.json_normalize(reduce_city)
city.head()

,Cidades,Correção,distance
0,Phila,Philadelphia,7
1,Philly,Philadelphia,7
2,Philadelphia,Philadelphia,0
3,Philadelphia.,Philadelphia,1
4,Pittsburgh,Pittsburgh,0
